# Toxicology Testing
For this assignment, you will use a chemical dataset to train a neural network to predict human reaction to exposure to certain compounds.

# Import Packages

In [8]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import deepchem as dc
from sklearn.metrics import accuracy_score
from tensorflow import keras
from keras import layers

# Set Environment

In [3]:
np.random.seed(456)
tf.random.set_seed(456)

# Load and Prep Data

In [4]:
_, (train, valid, test), _ = dc.molnet.load_tox21()
train_X, train_y, train_w = train.X, train.y, train.w
valid_X, valid_y, valid_w = valid.X, valid.y, valid.w
test_X, test_y, test_w = test.X, test.y, test.w
train_y = train_y[:, 0]
valid_y = valid_y[:, 0]
test_y = test_y[:, 0]
train_w = train_w[:, 0]
valid_w = valid_w[:, 0]
test_w = test_w[:, 0]

# Significant Modification
Code originally written for Tensorflow v1, which has been deprecated for years. For this assignment, we will use Tensorflow v2. Re-writing the entire architecture, paying attention to the steps in the assignment, rather than the starter code. Will also be using Keras sequential architectures. One hint comes from [here](https://www.geeksforgeeks.org/hidden-layer-perceptron-in-tensorflow/).

# Define Model Placeholders

In [10]:
d = 1024
n_hidden = 50
learning_rate = .001
n_epochs = 10
batch_size = 100

# Create Model Architecture

In [12]:
model = keras.Sequential([ 
	layers.Dense(256, activation = 'relu', input_shape = [d]), 
	layers.BatchNormalization(), 
	layers.Dense(256, activation = 'relu'), 
	layers.Dropout(0.3), 
	layers.BatchNormalization(), 
	layers.Dense(7, activation = 'sigmoid') 
]) 

model.compile( 
	loss = 'categorical_crossentropy', 
	optimizer = 'adam', 
	metrics = ['AUC'] 
) 

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               262400    
                                                                 
 batch_normalization_2 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 7)                

# Implement Mini-Batch Training

In [17]:
model.fit(train_X, train_y, 
          epochs = n_epochs, 
          batch_size = batch_size, 
          verbose = 1, 
          validation_data = (valid_X, valid_y)) 

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/lib/pyt

In [16]:
train_writer = tf.summary.FileWriter('/tmp/fcnet-tox21',
                                     tf.get_default_graph())
N = train_X.shape[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 0

for epoch in range(n_epochs):
    pos = 0

while N > pos:

      batch_X = train_X[pos:pos+batch_size]
      batch_y = train_y[pos:pos+batch_size]
      feed_dict = {x: batch_X, y: batch_y}
      _, summary, loss = sess.run([train_op, merged, l], feed_dict = feed_dict)
      print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
      train_writer.add_summary(summary, step)

      step += 1
      pos += batch_size

# Make Predictions
valid_y_pred = sess.run(y_pred, feed_dict={x: valid_X})

AttributeError: module 'tensorboard.summary._tf.summary' has no attribute 'FileWriter'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2c4d82d2-b129-4486-baa4-05c502f179ef' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>